In [1]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import pyodbc
import pandas_access as pa
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.experimental import enable_hist_gradient_boosting 
from sklearn.ensemble import RandomForestClassifier,HistGradientBoostingClassifier,VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, roc_auc_score,classification_report
from sklearn.model_selection import cross_val_predict,GridSearchCV, StratifiedKFold
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler,MinMaxScaler  
from sklearn.neural_network import MLPClassifier
from tensorflow import keras
import tensorflow as tf
import imblearn
from imblearn.over_sampling import SMOTE,BorderlineSMOTE,ADASYN,SVMSMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as imbpipeline
import seaborn as sns
from collections import Counter
import loras
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [2]:
pd.set_option('display.max_columns',200)
pd.set_option('display.max_rows',200)

In [3]:
dataset_close=pd.read_csv('data\hospital_closure\hospital_closure.csv')

In [4]:
MDB_2017='data\hospital_compare\hc_apr2017\Hospital.mdb'
MDB_2016 = 'data\hospital_compare\hc_may2016\Hospital.mdb'
MDB_2015='data\hospital_compare\Copy of HospitalMay2015.mdb'
MDB_2014='data\hospital_compare\HC_May2014.mdb'

DRV = '{Microsoft Access Driver (*.mdb, *.accdb)}'
PWD = 'pw'

con_2017 = pyodbc.connect('DRIVER={};DBQ={};PWD={}'.format(DRV,MDB_2017,PWD))
con_2016 = pyodbc.connect('DRIVER={};DBQ={};PWD={}'.format(DRV,MDB_2016,PWD))
con_2015 = pyodbc.connect('DRIVER={};DBQ={};PWD={}'.format(DRV,MDB_2015,PWD))
con_2014 = pyodbc.connect('DRIVER={};DBQ={};PWD={}'.format(DRV,MDB_2014,PWD))
cur_2017 = con_2017.cursor()
cur_2016 = con_2016.cursor()
cur_2015 = con_2015.cursor()
cur_2014 = con_2014.cursor()

In [5]:
table_name = 'HQI_HOSP_ReadmDeath'
table_name2='HQI_HOSP_ReadmCompDeath'
table_name3='HQI_HOSP_Comp'
query = "SELECT * FROM {}".format(table_name)
query2="SELECT * FROM {}".format(table_name2)
query3="SELECT * FROM {}".format(table_name3)
rows_2017 = cur_2017.execute(query).fetchall()
rows_2017_c=cur_2017.execute(query3).fetchall()
rows_2016 = cur_2016.execute(query).fetchall()
rows_2016_c = cur_2016.execute(query3).fetchall()
rows_2015 = cur_2015.execute(query2).fetchall()
#rows_2014 = cur_2014.execute(query2).fetchall()

In [40]:
list(cur_2015.tables())

[('data\\hospital_compare\\Copy of HospitalMay2015.mdb', None, 'MSysAccessStorage', 'SYSTEM TABLE', None),
 ('data\\hospital_compare\\Copy of HospitalMay2015.mdb', None, 'MSysAccessXML', 'SYSTEM TABLE', None),
 ('data\\hospital_compare\\Copy of HospitalMay2015.mdb', None, 'MSysACEs', 'SYSTEM TABLE', None),
 ('data\\hospital_compare\\Copy of HospitalMay2015.mdb', None, 'MSysIMEXColumns', 'SYSTEM TABLE', None),
 ('data\\hospital_compare\\Copy of HospitalMay2015.mdb', None, 'MSysIMEXSpecs', 'SYSTEM TABLE', None),
 ('data\\hospital_compare\\Copy of HospitalMay2015.mdb', None, 'MSysNameMap', 'SYSTEM TABLE', None),
 ('data\\hospital_compare\\Copy of HospitalMay2015.mdb', None, 'MSysNavPaneGroupCategories', 'SYSTEM TABLE', None),
 ('data\\hospital_compare\\Copy of HospitalMay2015.mdb', None, 'MSysNavPaneGroups', 'SYSTEM TABLE', None),
 ('data\\hospital_compare\\Copy of HospitalMay2015.mdb', None, 'MSysNavPaneGroupToObjects', 'SYSTEM TABLE', None),
 ('data\\hospital_compare\\Copy of HospitalMa

In [7]:
for row in cur_2015.columns(table='HQI_HOSP_ReadmCompDeath'):
    print(row.column_name)

Provider ID
Hospital Name
Measure Name
Measure ID
Compared to National
Denominator
Score
Lower Estimate
Higher Estimate
Footnote
Measure Start Date
Measure End Date


In [8]:
arr_2017=np.array(rows_2017)
dataset_2017=pd.DataFrame(arr_2017[:,[0,3,6]],columns=['id','score_name','Score'])
arr_2017_c=np.array(rows_2017_c)
dataset_2017_c=pd.DataFrame(arr_2017_c[:,[0,3,6]],columns=['id','score_name','Score'])
arr_2016=np.array(rows_2016)
dataset_2016=pd.DataFrame(arr_2016[:,[0,3,6]],columns=['id','score_name','Score'])
arr_2016_c=np.array(rows_2016_c)
dataset_2016_c=pd.DataFrame(arr_2016_c[:,[0,3,6]],columns=['id','score_name','Score'])
arr_2015=np.array(rows_2015)
dataset_2015=pd.DataFrame(arr_2015[:,[0,3,6]],columns=['id','score_name','Score'])
#dataset_2014=pd.DataFrame(arr_2014[:,[1,5,6,7,8,9,10]],columns=['id','1','2','3','4','5','6'])

In [9]:
dataset_2015=dataset_2015[dataset_2015['id'].map(lambda x: x.find('F')==-1)]
dataset_2015.drop(dataset_2015[dataset_2015.id=='670092'].index,inplace=True)
dataset_2016=dataset_2016[dataset_2016['id'].map(lambda x: x.find('F')==-1)]
dataset_2016_c=dataset_2016_c[dataset_2016_c['id'].map(lambda x: x.find('F')==-1)]

In [10]:
dataset_2016=pd.concat([dataset_2016,dataset_2016_c],axis=0)
dataset_2017=pd.concat([dataset_2017,dataset_2017_c],axis=0)

In [11]:
dataset_2017

,id,score_name,Score
0,010001,MORT_30_AMI,12.5
1,010001,MORT_30_CABG,4.2
2,010001,MORT_30_COPD,9.3
3,010001,MORT_30_HF,12.4
4,010001,MORT_30_PN,15.5
...,...,...,...
52850,670115,PSI_4_SURG_COMP,Not Available
52851,670115,PSI_6_IAT_PTX,Not Available
52852,670115,PSI_7_CVCBI,Not Available
52853,670115,PSI_8_POST_HIP,Not Available


In [12]:
dataset_2015.sort_values(by=['id','score_name'],inplace=True)
dataset_2016.sort_values(by=['id','score_name'],inplace=True)
dataset_2017.sort_values(by=['id','score_name'],inplace=True)

In [13]:
da_2015=dataset_2015[['id','Score']].to_numpy().reshape(4654,38)
da_2016=dataset_2016[['id','Score']].to_numpy().reshape(4643,42)
da_2017=dataset_2017[['id','Score']].to_numpy().reshape(4805,50)

In [14]:
arr1=[i*2 for i in range(19)]
arr2=[i*2 for i in range(21)]
arr3=[i*2 for i in range(25)]

In [15]:
da_2015=np.delete(da_2015,arr1[1:],axis=1)
da_2016=np.delete(da_2016,arr2[1:],axis=1)
da_2017=np.delete(da_2017,arr3[1:],axis=1)

In [16]:
da_2015=pd.DataFrame(da_2015,columns=['id','COMP_HIP_KNEE','MORT_30_AMI','MORT_30_COPD','MORT_30_HF','MORT_30_PN','MORT_30_STK','PSI_12_POSTOP_PULMEMB_DVT','PSI_14_POSTOP_DEHIS','PSI_15_ACC_LAC','PSI_4_SURG_COMP','PSI_6_IAT_PTX','PSI_90_SAFETY','READM_30_AMI','READM_30_COPD','READM_30_HF','READM_30_HIP_KNEE','READM_30_HOSP_WIDE','READM_30_PN','READM_30_STK'])
da_2016=pd.DataFrame(da_2016,columns=['id','COMP_HIP_KNEE','MORT_30_AMI','MORT_30_CABG','MORT_30_COPD','MORT_30_HF','MORT_30_PN','MORT_30_STK','PSI_12_POSTOP_PULMEMB_DVT','PSI_14_POSTOP_DEHIS','PSI_15_ACC_LAC','PSI_4_SURG_COMP','PSI_6_IAT_PTX','PSI_90_SAFETY','READM_30_AMI','READM_30_CABG','READM_30_COPD','READM_30_HF','READM_30_HIP_KNEE','READM_30_HOSP_WIDE','READM_30_PN','READM_30_STK'])
da_2017=pd.DataFrame(da_2017,columns=['id','COMP_HIP_KNEE','MORT_30_AMI','MORT_30_CABG','MORT_30_COPD','MORT_30_HF','MORT_30_PN','MORT_30_STK','PSI_12_POSTOP_PULMEMB_DVT','PSI_13_POST_SEPSIS','PSI_14_POSTOP_DEHIS','PSI_15_ACC_LAC','PSI_3_ULCER','PSI_4_SURG_COMP','PSI_6_IAT_PTX','PSI_7_CVCBI','PSI_8_POST_HIP','PSI_90_SAFETY','READM_30_AMI','READM_30_CABG','READM_30_COPD','READM_30_HF','READM_30_HIP_KNEE','READM_30_HOSP_WIDE','READM_30_PN','READM_30_STK'])

In [17]:
da_2016.drop(columns=['MORT_30_CABG','READM_30_CABG'],inplace=True)
da_2017.drop(columns=['MORT_30_CABG','PSI_13_POST_SEPSIS','PSI_3_ULCER','PSI_7_CVCBI','PSI_8_POST_HIP','READM_30_CABG'],inplace=True)


In [18]:
close_2016=dataset_close[dataset_close['year_close_combined']==2016]
close_2015=dataset_close[dataset_close['year_close_combined']==2015]
close_2014=dataset_close[dataset_close['year_close_combined']==2014]
da_2017['Closure']=da_2017.id.apply(lambda x:1 if int(x) in np.array(close_2016['id']) else 0)
da_2016['Closure']=da_2016.id.apply(lambda x:1 if int(x) in np.array(close_2015['id']) else 0)
da_2015['Closure']=da_2015.id.apply(lambda x:1 if int(x) in np.array(close_2014['id']) else 0)

In [19]:
da_2017['Closure'].value_counts(dropna=0)

0    4803
1       2
Name: Closure, dtype: int64

In [21]:
da_2017[da_2017['Closure']==1]

,id,COMP_HIP_KNEE,MORT_30_AMI,MORT_30_COPD,MORT_30_HF,MORT_30_PN,MORT_30_STK,PSI_12_POSTOP_PULMEMB_DVT,PSI_14_POSTOP_DEHIS,PSI_15_ACC_LAC,PSI_4_SURG_COMP,PSI_6_IAT_PTX,PSI_90_SAFETY,READM_30_AMI,READM_30_COPD,READM_30_HF,READM_30_HIP_KNEE,READM_30_HOSP_WIDE,READM_30_PN,READM_30_STK,Closure
4030,450214,Not Available,Not Available,8.3,11.2,16.5,Not Available,4.72,2.29,1.24,Not Available,0.40,0.82,Not Available,20.2,20.1,Not Available,15.8,15.9,Not Available,1
4101,450605,Not Available,Not Available,8.2,10.7,15.2,Not Available,Not Available,Not Available,1.69,Not Available,0.47,0.96,Not Available,18.4,21.3,Not Available,14.7,15.1,Not Available,1


In [22]:
da_2016[da_2016['Closure']==1]

,id,COMP_HIP_KNEE,MORT_30_AMI,MORT_30_COPD,MORT_30_HF,MORT_30_PN,MORT_30_STK,PSI_12_POSTOP_PULMEMB_DVT,PSI_14_POSTOP_DEHIS,PSI_15_ACC_LAC,PSI_4_SURG_COMP,PSI_6_IAT_PTX,PSI_90_SAFETY,READM_30_AMI,READM_30_COPD,READM_30_HF,READM_30_HIP_KNEE,READM_30_HOSP_WIDE,READM_30_PN,READM_30_STK,Closure
1092,140026,2.9,13.6,7.8,10.7,12.2,15.2,4.40,1.60,1.06,Not Available,0.36,0.64,17.2,20.2,20.8,4.7,15.5,17.6,12.1,1
1492,170010,Not Available,Not Available,7.8,13.1,12.3,15.4,4.85,Not Available,1.85,Not Available,0.38,0.84,Not Available,19.2,20.5,Not Available,15.8,15.1,11.9,1
1675,180117,Not Available,Not Available,7.3,12.0,13.4,Not Available,3.80,Not Available,1.58,Not Available,0.38,0.72,Not Available,19.7,24.2,Not Available,15.2,17.8,Not Available,1
2265,250084,3.6,15.1,8.5,13.7,16.1,16.0,3.80,1.64,1.65,Not Available,0.39,0.76,18.1,20.5,22.2,4.7,14.9,16.9,13.5,1
2368,260116,4.4,Not Available,5.8,10.6,13.7,15.1,4.73,1.64,1.79,Not Available,0.37,0.81,Not Available,21.4,22.7,5.4,15.6,18.0,Not Available,1
2582,290020,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,1.80,Not Available,0.39,0.81,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,1
3250,370051,Not Available,Not Available,6.8,10.2,10.8,Not Available,Not Available,Not Available,1.75,Not Available,0.38,0.80,Not Available,20.0,20.9,Not Available,15.3,17.0,Not Available,1
3656,420054,Not Available,Not Available,8.3,11.5,11.9,Not Available,Not Available,Not Available,1.65,Not Available,0.38,0.85,Not Available,20.1,21.4,Not Available,15.5,17.5,Not Available,1
3850,450005,Not Available,Not Available,8.0,13.0,10.9,Not Available,3.71,1.66,1.23,Not Available,0.46,0.65,Not Available,18.7,20.8,Not Available,15.0,16.0,Not Available,1
3983,450497,Not Available,Not Available,7.7,13.8,11.1,Not Available,Not Available,Not Available,1.57,Not Available,0.38,0.75,Not Available,19.0,22.1,Not Available,15.1,17.9,Not Available,1


In [23]:
da_2015[da_2015['Closure']==1]

,id,COMP_HIP_KNEE,MORT_30_AMI,MORT_30_COPD,MORT_30_HF,MORT_30_PN,MORT_30_STK,PSI_12_POSTOP_PULMEMB_DVT,PSI_14_POSTOP_DEHIS,PSI_15_ACC_LAC,PSI_4_SURG_COMP,PSI_6_IAT_PTX,PSI_90_SAFETY,READM_30_AMI,READM_30_COPD,READM_30_HF,READM_30_HIP_KNEE,READM_30_HOSP_WIDE,READM_30_PN,READM_30_STK,Closure
205,040042,Not Available,14.2,9.4,11.3,14.1,16.2,4.67,1.80,1.81,Not Available,0.37,0.80,Not Available,22.1,23.1,Not Available,17.2,19.3,12.6,1
443,050435,2.8,Not Available,8.0,10.4,11.6,Not Available,4.11,1.87,2.10,Not Available,0.39,0.84,Not Available,20.4,22.6,4.3,15.7,17.7,Not Available,1
834,100239,3.7,15.0,7.1,12.5,11.0,14.1,4.43,1.86,1.53,103.97,0.42,0.73,Not Available,19.6,22.3,5.6,15.4,16.9,14.0,1
977,110183,Not Available,Not Available,7.0,12.9,12.2,Not Available,3.82,1.87,1.47,Not Available,0.39,0.70,Not Available,21.0,23.2,Not Available,15.0,17.3,Not Available,1
1725,190009,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,4.54,Not Available,2.19,Not Available,0.41,0.92,Not Available,Not Available,Not Available,Not Available,16.1,17.9,Not Available,1
1941,220067,3.9,14.9,6.2,11.0,9.6,16.1,2.57,2.11,1.60,107.28,0.66,0.82,18.1,21.0,24.4,6.2,15.0,19.7,14.1,1
3823,440174,Not Available,Not Available,7.1,11.1,11.1,Not Available,Not Available,Not Available,1.88,Not Available,0.41,0.86,Not Available,21.3,23.6,Not Available,15.0,18.6,Not Available,1
3971,450373,Not Available,Not Available,7.4,11.2,10.2,Not Available,Not Available,Not Available,1.89,Not Available,0.46,0.87,Not Available,19.4,23.8,Not Available,15.6,17.7,Not Available,1
4108,450884,Not Available,Not Available,7.3,11.2,11.2,Not Available,Not Available,Not Available,1.83,Not Available,0.40,0.84,Not Available,19.6,22.6,Not Available,14.0,16.7,Not Available,1
4603,670002,Not Available,Not Available,6.9,11.4,13.0,Not Available,3.99,Not Available,1.45,Not Available,0.39,0.72,Not Available,20.0,22.2,Not Available,15.9,17.0,Not Available,1


In [24]:
da_2017=da_2017.replace({'Not Available':np.nan,' Not Available':np.nan})
da_2016=da_2016.replace({'Not Available':np.nan,' Not Available':np.nan})
da_2015=da_2015.replace({'Not Available':np.nan,' Not Available':np.nan})

In [68]:
da_=pd.concat([da_2017,da_2016,da_2015])
da_.drop(columns=['id'],inplace=True)

In [69]:
for col in da_:
    da_[col]=pd.to_numeric(da_[col])

In [53]:
da_

,COMP_HIP_KNEE,MORT_30_AMI,MORT_30_COPD,MORT_30_HF,MORT_30_PN,MORT_30_STK,PSI_12_POSTOP_PULMEMB_DVT,PSI_14_POSTOP_DEHIS,PSI_15_ACC_LAC,PSI_4_SURG_COMP,PSI_6_IAT_PTX,PSI_90_SAFETY,READM_30_AMI,READM_30_COPD,READM_30_HF,READM_30_HIP_KNEE,READM_30_HOSP_WIDE,READM_30_PN,READM_30_STK,Closure
0,3.8,12.5,9.3,12.4,15.5,15.4,3.32,2.72,1.35,168.26,0.36,0.68,16.5,21.1,21.4,5.1,15.4,18.7,12.7,0
1,3.0,16.0,7.6,15.5,20.8,15.5,5.63,2.11,1.25,179.05,0.47,0.85,16.7,18.0,21.9,5.7,14.9,16.4,13.4,0
2,3.8,16.7,7.1,15.6,18.2,17.9,3.85,2.50,1.93,198.33,0.41,0.91,16.1,19.8,20.6,5.0,15.4,17.9,12.0,0
3,NaN,NaN,9.3,14.4,18.8,16.6,4.57,NaN,1.20,NaN,0.40,0.79,NaN,19.9,21.1,NaN,16.6,17.3,12.7,0
4,NaN,NaN,8.2,12.7,15.7,NaN,NaN,NaN,1.41,NaN,0.41,0.90,NaN,19.2,23.1,NaN,15.7,16.0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4649,NaN,NaN,NaN,NaN,NaN,NaN,4.44,NaN,1.76,NaN,0.40,0.92,NaN,NaN,NaN,NaN,14.9,NaN,NaN,0
4650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4651,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4652,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [94]:
list(da_.columns[:-1])

['COMP_HIP_KNEE',
 'MORT_30_AMI',
 'MORT_30_COPD',
 'MORT_30_HF',
 'MORT_30_PN',
 'MORT_30_STK',
 'PSI_12_POSTOP_PULMEMB_DVT',
 'PSI_14_POSTOP_DEHIS',
 'PSI_15_ACC_LAC',
 'PSI_4_SURG_COMP',
 'PSI_6_IAT_PTX',
 'PSI_90_SAFETY',
 'READM_30_AMI',
 'READM_30_COPD',
 'READM_30_HF',
 'READM_30_HIP_KNEE',
 'READM_30_HOSP_WIDE',
 'READM_30_PN',
 'READM_30_STK']

In [34]:
da_.describe()

,COMP_HIP_KNEE,MORT_30_AMI,MORT_30_COPD,MORT_30_HF,MORT_30_PN,MORT_30_STK,PSI_12_POSTOP_PULMEMB_DVT,PSI_14_POSTOP_DEHIS,PSI_15_ACC_LAC,PSI_4_SURG_COMP,PSI_6_IAT_PTX,PSI_90_SAFETY,READM_30_AMI,READM_30_COPD,READM_30_HF,READM_30_HIP_KNEE,READM_30_HOSP_WIDE,READM_30_PN,READM_30_STK,Closure
count,8233.000000,7282.000000,10900.000000,10995.000000,12275.000000,8284.000000,9036.000000,7751.000000,9631.000000,5502.000000,9650.000000,9706.000000,6632.000000,11103.000000,11212.000000,8266.000000,13305.000000,12309.000000,8109.000000,14102.000000
mean,3.185461,14.363142,7.913459,11.959345,13.345222,15.044532,4.518752,1.978051,1.726129,124.304022,0.400696,0.848702,17.252141,20.345375,22.234267,4.895717,15.470943,17.133959,12.886496,0.001560
std,0.609817,1.343128,1.067743,1.469456,2.866427,1.712835,1.722131,0.435237,0.644749,20.741338,0.072136,0.183101,1.119150,1.313005,1.610627,0.664587,0.889226,1.292103,1.174144,0.039468
min,1.400000,9.400000,4.600000,6.000000,6.400000,8.600000,0.860000,0.860000,0.260000,56.260000,0.180000,0.370000,13.100000,15.500000,15.800000,2.400000,10.800000,12.900000,8.700000,0.000000
25%,2.800000,13.500000,7.200000,11.000000,11.100000,13.900000,3.430000,1.650000,1.300000,109.320000,0.360000,0.730000,16.500000,19.500000,21.200000,4.500000,14.900000,16.300000,12.100000,0.000000
50%,3.100000,14.300000,7.800000,11.900000,12.800000,14.900000,4.200000,1.880000,1.630000,123.605000,0.390000,0.830000,17.200000,20.300000,22.100000,4.800000,15.400000,17.000000,12.800000,0.000000
75%,3.500000,15.200000,8.500000,12.900000,15.400000,16.100000,5.200000,2.230000,2.020000,137.985000,0.430000,0.930000,18.000000,21.100000,23.200000,5.300000,15.900000,17.900000,13.500000,0.000000
max,6.900000,20.600000,14.100000,18.500000,26.800000,23.800000,20.880000,4.980000,7.220000,212.160000,0.880000,2.140000,21.700000,28.000000,31.700000,9.400000,21.400000,24.700000,18.500000,1.000000


In [35]:
da_[da_['Closure']==0].describe()

,COMP_HIP_KNEE,MORT_30_AMI,MORT_30_COPD,MORT_30_HF,MORT_30_PN,MORT_30_STK,PSI_12_POSTOP_PULMEMB_DVT,PSI_14_POSTOP_DEHIS,PSI_15_ACC_LAC,PSI_4_SURG_COMP,PSI_6_IAT_PTX,PSI_90_SAFETY,READM_30_AMI,READM_30_COPD,READM_30_HF,READM_30_HIP_KNEE,READM_30_HOSP_WIDE,READM_30_PN,READM_30_STK,Closure
count,8227.000000,7277.000000,10880.000000,10975.000000,12255.000000,8277.000000,9022.000000,7741.000000,9609.000000,5500.000000,9628.000000,9684.000000,6629.000000,11083.000000,11192.000000,8260.000000,13284.000000,12288.000000,8103.000000,14080.0
mean,3.185195,14.363007,7.914136,11.959863,13.346756,15.044195,4.519319,1.978237,1.726271,124.310815,0.400673,0.848822,17.251893,20.345836,22.234257,4.895533,15.471063,17.133651,12.886388,0.0
std,0.609775,1.343495,1.068041,1.470024,2.867525,1.713404,1.723253,0.435420,0.645355,20.742026,0.072156,0.183253,1.119306,1.313559,1.611232,0.664566,0.889586,1.292318,1.174329,0.0
min,1.400000,9.400000,4.600000,6.000000,6.400000,8.600000,0.860000,0.860000,0.260000,56.260000,0.180000,0.370000,13.100000,15.500000,15.800000,2.400000,10.800000,12.900000,8.700000,0.0
25%,2.800000,13.500000,7.200000,11.000000,11.100000,13.900000,3.430000,1.650000,1.300000,109.327500,0.360000,0.730000,16.500000,19.500000,21.200000,4.500000,14.900000,16.300000,12.100000,0.0
50%,3.100000,14.300000,7.800000,11.900000,12.800000,14.900000,4.200000,1.880000,1.630000,123.610000,0.390000,0.830000,17.200000,20.300000,22.100000,4.800000,15.400000,17.000000,12.800000,0.0
75%,3.500000,15.200000,8.500000,12.900000,15.400000,16.100000,5.200000,2.230000,2.020000,137.990000,0.430000,0.930000,18.000000,21.100000,23.200000,5.300000,15.900000,17.900000,13.500000,0.0
max,6.900000,20.600000,14.100000,18.500000,26.800000,23.800000,20.880000,4.980000,7.220000,212.160000,0.880000,2.140000,21.700000,28.000000,31.700000,9.400000,21.400000,24.700000,18.500000,0.0


In [36]:
da_[da_['Closure']==1].describe()

,COMP_HIP_KNEE,MORT_30_AMI,MORT_30_COPD,MORT_30_HF,MORT_30_PN,MORT_30_STK,PSI_12_POSTOP_PULMEMB_DVT,PSI_14_POSTOP_DEHIS,PSI_15_ACC_LAC,PSI_4_SURG_COMP,PSI_6_IAT_PTX,PSI_90_SAFETY,READM_30_AMI,READM_30_COPD,READM_30_HF,READM_30_HIP_KNEE,READM_30_HOSP_WIDE,READM_30_PN,READM_30_STK,Closure
count,6.000000,5.000000,20.000000,20.000000,20.000000,7.000000,14.000000,10.000000,22.000000,2.000000,22.000000,22.000000,3.000000,20.000000,20.000000,6.000000,21.000000,21.000000,6.000000,22.0
mean,3.550000,14.560000,7.545000,11.675000,12.405000,15.442857,4.152857,1.834000,1.664091,105.625000,0.410909,0.795909,17.800000,20.090000,22.240000,5.150000,15.395238,17.314286,13.033333,1.0
std,0.609098,0.642651,0.831913,1.106381,1.912968,0.741299,0.606737,0.223318,0.271252,2.340523,0.063088,0.080573,0.519615,0.947518,1.258403,0.706399,0.631250,1.173578,0.962635,0.0
min,2.800000,13.600000,5.800000,10.200000,9.600000,14.100000,2.570000,1.600000,1.060000,103.970000,0.360000,0.640000,17.200000,18.400000,20.100000,4.300000,14.000000,15.100000,11.900000,1.0
25%,3.075000,14.200000,7.075000,10.925000,11.075000,15.150000,3.805000,1.645000,1.540000,104.797500,0.380000,0.735000,17.650000,19.550000,21.200000,4.700000,15.000000,16.900000,12.225000,1.0
50%,3.650000,14.900000,7.550000,11.250000,12.050000,15.400000,4.255000,1.830000,1.670000,105.625000,0.390000,0.810000,18.100000,20.050000,22.250000,5.050000,15.400000,17.500000,13.050000,1.0
75%,3.850000,15.000000,8.050000,12.600000,13.475000,16.050000,4.637500,1.870000,1.825000,106.452500,0.410000,0.840000,18.100000,20.625000,23.125000,5.550000,15.700000,17.900000,13.875000,1.0
max,4.400000,15.100000,9.400000,13.800000,16.500000,16.200000,4.850000,2.290000,2.190000,107.280000,0.660000,0.960000,18.100000,22.100000,24.400000,6.200000,17.200000,19.700000,14.100000,1.0


In [37]:
da_.drop(columns=['PSI_4_SURG_COMP','READM_30_AMI'],inplace=True)

In [62]:
da_.fillna(da_.median(),inplace=True)

In [70]:
dataset_=da_.to_numpy()

In [71]:
imp_mean = IterativeImputer()
imp_mean.fit(dataset_)
b=imp_mean.transform(dataset_)
dataset_=b

In [72]:
X_train, X_test, y_train, y_test = train_test_split(dataset_[:,:-1],
                                                    dataset_[:,-1],
                                                    test_size=0.3,
                                                    stratify=dataset_[:,-1]
                                                    )

In [73]:
pipeline = imbpipeline(steps = [['smote', SMOTE()],
                                
                                ['classifier', RandomForestClassifier()]])

stratified_kfold = StratifiedKFold(n_splits=5,
                                       shuffle=True
                                       )
    
param_grid = {'classifier__n_estimators':[10,20,30,40,50,60,70,80,90,100,150,200,300,400,500]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_RF_s = grid_search.best_score_
test_score_RF_s = grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_RF_s}\nTest score: {test_score_RF_s}')

Cross-validation score: 0.26666666666666666
Test score: 0.0


In [75]:
pipeline = imbpipeline(steps = [['ADASYN', ADASYN()],
                                
                                ['classifier', RandomForestClassifier()]])

stratified_kfold = StratifiedKFold(n_splits=5,
                                       shuffle=True
                                       )
    
param_grid = {'classifier__n_estimators':[100,200]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_RF_a = grid_search.best_score_
test_score_RF_a = grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_RF_a}\nTest score: {test_score_RF_a}')

Cross-validation score: 0.18
Test score: 0.0


In [77]:
pipeline = imbpipeline(steps = [['SVMSMOTE', SVMSMOTE()],
                                
                                ['classifier', RandomForestClassifier()]])

stratified_kfold = StratifiedKFold(n_splits=5,
                                       shuffle=True
                                       )
    
param_grid = {'classifier__n_estimators':[10,20,30,40,50,60,70,80,90,100,150,200,300,400,500]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_RF_svm = grid_search.best_score_
test_score_RF_svm = grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_RF_svm}\nTest score: {test_score_RF_svm}')

Cross-validation score: 0.16666666666666666
Test score: 0.0


In [78]:
pipeline = imbpipeline(steps = [['SMOTE', SMOTE()],
                                
                                ['classifier', LogisticRegression()]])
    
param_grid = {'classifier__C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_LR_s = grid_search.best_score_
test_score_LR_s= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_LR_s}\nTest score: {test_score_LR_s}')

Cross-validation score: 0.01333079130502226
Test score: 0.016129032258064516


C:\Users\zys\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [79]:
pipeline = imbpipeline(steps = [['ADASYN', ADASYN()],
                                
                                ['classifier', DecisionTreeClassifier()]])
    
param_grid = {'classifier__criterion':['gini','entropy'],'classifier__max_depth':[5,200,500]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_DT_a = grid_search.best_score_
test_score_DT_a= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_DT_a}\nTest score: {test_score_DT_a}')

Cross-validation score: 0.14190476190476192
Test score: 0.0


In [80]:
pipeline = imbpipeline(steps = [['SVMSMOTE',SVMSMOTE()],
                                
                                ['classifier', DecisionTreeClassifier()]])
    
param_grid = {'classifier__criterion':['gini','entropy'],'classifier__max_depth':[5,200,500]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_DT_svm = grid_search.best_score_
test_score_DT_svm= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_DT_svm}\nTest score: {test_score_DT_svm}')

Cross-validation score: 0.08094567970728962
Test score: 0.04819277108433735


In [81]:
pipeline = imbpipeline(steps = [['ADASYN', ADASYN()],
                                
                                ['classifier', XGBClassifier()]])

grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_xgb_a = grid_search.best_score_
test_score_xgb_a= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_xgb_a}\nTest score: {test_score_xgb_a}')

C:\Users\zys\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[06:51:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "criterion" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[06:51:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Cross-validation score: 0.4076190476190476
Test score: 0.0


In [83]:
pipeline = imbpipeline(steps = [['SVMSMOTE', SVMSMOTE()],
                                
                                ['classifier', XGBClassifier()]])
param_grid = {'classifier__min_child_weight': [1, 5, 10]
        }    
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_xgb_svm = grid_search.best_score_
test_score_xgb_svm= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_xgb_svm}\nTest score: {test_score_xgb_svm}')

C:\Users\zys\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[06:52:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Cross-validation score: 0.24000000000000005
Test score: 0.0


In [87]:
pipeline = imbpipeline(steps = [['SMOTE', SMOTE()],
                                
                                ['classifier', DecisionTreeClassifier()]])
    
param_grid = {'classifier__criterion':['gini','entropy'],'classifier__max_depth':[5,200,500]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_DT_s = grid_search.best_score_
test_score_DT_s= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_DT_s}\nTest score: {test_score_DT_s}')

Cross-validation score: 0.12207792207792209
Test score: 0.0


In [88]:
pipeline = imbpipeline(steps = [['ADASYN', ADASYN()],
                                
                                ['classifier', LogisticRegression()]])
    
param_grid = {'classifier__C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_LR_a = grid_search.best_score_
test_score_LR_a= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_LR_a}\nTest score: {test_score_LR_a}')

Cross-validation score: 0.011752259178918105
Test score: 0.016238159675236806


C:\Users\zys\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [89]:
pipeline = imbpipeline(steps = [['SVMSMOTE', SVMSMOTE()],
                                
                                ['classifier', LogisticRegression()]])
    
param_grid = {'classifier__C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_LR_svm = grid_search.best_score_
test_score_LR_svm= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_LR_svm}\nTest score: {test_score_LR_svm}')

Cross-validation score: 0.009108819811306198
Test score: 0.007858546168958742


C:\Users\zys\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [74]:
pipeline = imbpipeline(steps = [['SMOTE',SMOTE()],
                                
                                ['classifier', XGBClassifier()]])
    
param_grid = {'classifier__min_child_weight': [1, 5, 10]
        }
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_xgb_s = grid_search.best_score_
test_score_xgb_s= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_xgb_s}\nTest score: {test_score_xgb_s}')

C:\Users\zys\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[06:48:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Cross-validation score: 0.360952380952381
Test score: 0.30769230769230765


In [222]:
pipeline = imbpipeline(steps = [['SMOTE', SMOTE()],
                                
                                ['classifier', HistGradientBoostingClassifier()]])
    
param_grid = {'classifier__min_samples_leaf': [1,2,3,4, 5, 10]
        }
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_xgb_s = grid_search.best_score_
test_score_xgb_s= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_xgb_s}\nTest score: {test_score_xgb_s}')

Cross-validation score: 0.28380952380952384
Test score: 0.1111111111111111


In [227]:
voting=VotingClassifier(estimators=[('rf',RandomForestClassifier()),('xg',XGBClassifier()),('dt',DecisionTreeClassifier()),('hg',HistGradientBoostingClassifier())])
pipeline = imbpipeline(steps = [['SMOTE', SMOTE()],
                                
                                ['classifier', voting]])
    
param_grid = {'classifier__rf__n_estimators':[10,20,30,40,50,60,70,80,90,100,150,200,300,400,500],'classifier__xg__min_child_weight': [1, 5, 10],'classifier__hg__min_samples_leaf': [1, 5, 10]
        }
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_xgb_s = grid_search.best_score_
test_score_xgb_s= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_xgb_s}\nTest score: {test_score_xgb_s}')

C:\Users\zys\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[05:27:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Cross-validation score: 0.26
Test score: 0.2


In [226]:
voting=VotingClassifier(estimators=[('rf',RandomForestClassifier()),('xg',XGBClassifier()),('dt',DecisionTreeClassifier()),('hg',HistGradientBoostingClassifier())])
pipeline = imbpipeline(steps = [['SMOTE', SMOTE()],
                                
                                ['classifier', voting]])
voting.get_params().keys()

dict_keys(['estimators', 'flatten_transform', 'n_jobs', 'verbose', 'voting', 'weights', 'rf', 'xg', 'dt', 'hg', 'rf__bootstrap', 'rf__ccp_alpha', 'rf__class_weight', 'rf__criterion', 'rf__max_depth', 'rf__max_features', 'rf__max_leaf_nodes', 'rf__max_samples', 'rf__min_impurity_decrease', 'rf__min_impurity_split', 'rf__min_samples_leaf', 'rf__min_samples_split', 'rf__min_weight_fraction_leaf', 'rf__n_estimators', 'rf__n_jobs', 'rf__oob_score', 'rf__random_state', 'rf__verbose', 'rf__warm_start', 'xg__objective', 'xg__use_label_encoder', 'xg__base_score', 'xg__booster', 'xg__colsample_bylevel', 'xg__colsample_bynode', 'xg__colsample_bytree', 'xg__gamma', 'xg__gpu_id', 'xg__importance_type', 'xg__interaction_constraints', 'xg__learning_rate', 'xg__max_delta_step', 'xg__max_depth', 'xg__min_child_weight', 'xg__missing', 'xg__monotone_constraints', 'xg__n_estimators', 'xg__n_jobs', 'xg__num_parallel_tree', 'xg__random_state', 'xg__reg_alpha', 'xg__reg_lambda', 'xg__scale_pos_weight', 'xg__

In [90]:
models = pd.DataFrame({
    'Model': ['Random Forest', 'Logistic Regression', 
              'Decision Tree',  
              'Gradient Boosting'],
    'SMOTE': [cv_score_RF_s,cv_score_LR_s,cv_score_DT_s,cv_score_xgb_s],
    'ADASYN': [cv_score_RF_a,cv_score_LR_a,cv_score_DT_a,cv_score_xgb_a],
    'SVMSMOTE':[cv_score_RF_svm,cv_score_LR_svm,cv_score_DT_svm,cv_score_xgb_svm]})

In [91]:
models

,Model,SMOTE,ADASYN,SVMSMOTE
0,Random Forest,0.266667,0.180000,0.166667
1,Logistic Regression,0.013331,0.011752,0.009109
2,Decision Tree,0.122078,0.141905,0.080946
3,Gradient Boosting,0.360952,0.407619,0.240000
